# Chemical database initialization

### Goal

Bootstrap a chemical database with ~720,000 structures from the US EPA CompTox Dashboard's public dataset.

Set up the database so that it can be used for substructure searching via the [RDKit PostgreSQL database cartridge](http://www.rdkit.org/docs/Cartridge.html). Also include CASRNs and PubChem CIDs as much as possible.


### Data sources

`dsstox_20160701.tsv`
  - Downloaded from: https://comptox.epa.gov/dashboard/downloads (zip filename: DSSTox_Mapping_20160701.zip)
  - Date: 2016-07-01 (file generated); 2016-12-14 (posted on EPA website)
  - Accessed: 2017-01-05
  - "The DSSTOX mapping file contains mappings between the DSSTox substance identifier (DTXSID) and the associated InChI String and InChI Key."

`Dsstox_CAS_number_name.xlsx`
  - Downloaded from: https://comptox.epa.gov/dashboard/downloads
  - Date: 2016-11-14
  - Accessed: 2017-01-05
  - "The DSSTox Identifiers file is in Excel format and includes the CAS Number, DSSTox substance identifier (DTXSID) and the Preferred Name."

`PubChem_DTXSID_mapping_file.txt`
  - Downloaded from: https://comptox.epa.gov/dashboard/downloads
  - Date: 2016-11-14
  - Accessed: 2017-01-05
  - "The DSSTox to PubChem Identifiers mapping file is in TXT format and includes the PubChem SID, PubChem CID and DSSTox substance identifier (DTXSID)."


### Notes on software dependencies

Requires:

- A running instance of PostgreSQL and an empty database instantiated with the RDKit extension activated ([instructions](https://github.com/rdkit/rdkit/blob/master/Docs/Book/Install.md));
- A `commongroups` environment containing a configuration file (see docs, TBD);
- Python packages and dependencies: commongroups, rdkit, sqlalchemy, psycopg2, pandas.

## Imports & environment

In [1]:
import sys
from os.path import join as pjoin

import pandas as pd
from pandas import DataFrame

from rdkit import Chem
from rdkit.Chem import AllChem

from sqlalchemy import create_engine, MetaData, Table, Column, types
from sqlalchemy.sql import text

sys.path.append('..')
from commongroups.env import CommonEnv

### Database connection and file locations

Read config variables from a `commongroups` environment.

In [2]:
env = CommonEnv('setup')  # Or whatever local environment exists...
cfg = env.get_config()
con = create_engine(cfg['database_url'])

DATA_PATH = cfg['data_sources_path']

DTX_STRUCT = pjoin(DATA_PATH, 'dsstox_20160701.tsv')
DTX_CASRNS = pjoin(DATA_PATH, 'Dsstox_CAS_number_name.xlsx')
DTX_CIDS = pjoin(DATA_PATH, 'PubChem_DTXSID_mapping_file.txt')

2017-04-10 18:15:37,396 commongroups.env INFO Project path: /opt/akokai/data/cmg/setup


## Create database tables

In [3]:
create_tables = text(
    """
    CREATE TABLE dsstox (
        dtxsid text PRIMARY KEY,
        inchi text NOT NULL,
        inchikey text NOT NULL,
        bin bytea NOT NULL
    );

    CREATE TABLE dtx_casrn (
        casrn text,
        dtxsid text REFERENCES dsstox,
        name text,
        PRIMARY KEY (dtxsid, casrn)
    );

    CREATE TABLE dtx_cid (
        cid text,
        dtxsid text REFERENCES dsstox,
        PRIMARY KEY (dtxsid, cid)
    );
    """
)

res = con.execute(create_tables)

## Generate table of binary structural representations

- Take the list of EPA InChI(Key)s and DSSTox substance IDs, convert each InChI into a RDKit `Mol` object. Then convert each `Mol` into its binary representation. Put this into a PGSQL database table.

- Then update the table to create RDKit `mol` objects in a new column, using the binary representations and the `mol_from_pkl` function.

- This binary thing is a necessary intermediate step because there is no `mol_from_inchi` method in the PGSQL RDKit extension, there is only `mol_from_smiles`. (Otherwise we could go straight from InChI to molecules in the SQL table.)

### Notes

- The 720K rows seems to be too much to process in memory all at once, so we go through the file lazily in chunks.

- RDKit will fail to create many of the molecules from InChI because of very specific errors. The number of molecules we have in the end will probably be less than 720K.

- This will take a while and use lots of CPU and memory.

In [4]:
dtypes = {'dtxsid': types.Text,
          'inchi': types.Text,
          'inchikey': types.Text,
          'bin': types.Binary}

ninput = 719996
ncreated = 0
chunk = 10000

dtx = pd.read_table(DTX_STRUCT, names=['dtxsid', 'inchi', 'inchikey'],
                    chunksize=chunk, low_memory=True)

for df in dtx:
    df['mol'] = df.inchi.apply(Chem.MolFromInchi)
    df.dropna(inplace=True)
    n = len(df)
    ncreated += n
    print('{0} molecules created, {1} errors'.format(n, chunk - n))
    df['bin'] = df.mol.apply(lambda m: m.ToBinary())
    df.drop('mol', axis=1, inplace=True)
    df.to_sql('dsstox', con, if_exists='append', index=False, chunksize=65536, dtype=dtypes)

print('Total: {0} molecules created, {1} errors'.format(ncreated, ninput - ncreated))

9996 molecules created, 4 errors
9985 molecules created, 15 errors
9994 molecules created, 6 errors
9995 molecules created, 5 errors
9995 molecules created, 5 errors
9993 molecules created, 7 errors
9997 molecules created, 3 errors
9989 molecules created, 11 errors
9996 molecules created, 4 errors
9993 molecules created, 7 errors
9989 molecules created, 11 errors
9993 molecules created, 7 errors
9992 molecules created, 8 errors
9988 molecules created, 12 errors
10000 molecules created, 0 errors
10000 molecules created, 0 errors
9998 molecules created, 2 errors
9996 molecules created, 4 errors
9994 molecules created, 6 errors
9993 molecules created, 7 errors
9998 molecules created, 2 errors
10000 molecules created, 0 errors
10000 molecules created, 0 errors
10000 molecules created, 0 errors
10000 molecules created, 0 errors
9999 molecules created, 1 errors
9998 molecules created, 2 errors
9996 molecules created, 4 errors
9990 molecules created, 10 errors
9998 molecules created, 2 errors

### Generate `mol`-type column

In [5]:
update_molecules = text(
    """
    ALTER TABLE dsstox ADD COLUMN molecule mol;
    
    UPDATE dsstox SET molecule = mol_from_pkl(bin);
    """
)
res = con.execute(update_molecules)
print(res.rowcount, 'rows changed')

719631 rows changed


In [6]:
# Check results.
try:
    assert res.rowcount == ncreated
except AssertionError:
    print('Wrong number of molecules!')    

In [7]:
tidy_molecules = text(
    """
    ALTER TABLE dsstox ALTER COLUMN molecule SET NOT NULL;
    
    ALTER TABLE dsstox DROP COLUMN bin;
    """
)
res = con.execute(tidy_molecules)

In [8]:
# Check that the table contains expected data... 
cmd = text('select * from dsstox limit 5;')
con.execute(cmd).fetchall()

[('DTXSID8051382', 'InChI=1S/Li.2H2O/h;2*1H2/q+1;;/p-1', 'GLXDVVHUTZTUQK-UHFFFAOYSA-M', 'O.[Li+].[OH-]'),
 ('DTXSID5058301', 'InChI=1S/Ag.O', 'OTCVAHKKMMUFAY-UHFFFAOYSA-N', '[Ag].[O]'),
 ('DTXSID4065739', 'InChI=1S/Pb/i1+3', 'WABPQHHGFIMREM-AKLPVKDBSA-N', '[210Pb]'),
 ('DTXSID8066630', 'InChI=1S/Hg.Se', 'YQMLDSWXEQOSPP-UHFFFAOYSA-N', '[Hg].[Se]'),
 ('DTXSID9067437', 'InChI=1S/Sb.Tl', 'YJLNSAVOCPBJTN-UHFFFAOYSA-N', '[Sb].[Tl]')]

## Create DataFrame of DTXSIDs for which we have molecules

This will be used to avoid violations of foreign key constraints...

In [9]:
ids = pd.read_sql("SELECT dtxsid FROM dsstox", con)

## Import external ID mappings: DTXSID to CASRN, CID

### Load DTXSID:CASRN mappings

Note that these are all 1:1 mappings.

In [10]:
dtx_cas_data = pd.read_excel(DTX_CASRNS)
cas_cols = ['casrn', 'dtxsid', 'name']
dtx_cas_data.columns = cas_cols
print(len(dtx_cas_data), 'DTXSID:CASRN mappings in data source')

753398 DTXSID:CASRN mappings in data source


In [11]:
# Filter mappings to include only DTXSIDs for which we already have a molecule.
dtx_cas_data = dtx_cas_data.loc[dtx_cas_data['dtxsid'].isin(ids['dtxsid'])]
print(len(dtx_cas_data), 'DTXSID:CASRN mappings available to add to database')

719631 DTXSID:CASRN mappings available to add to database


In [12]:
dtypes_cas = dict(zip(cas_cols, 3*[types.Text]))
dtx_cas_data.to_sql('dtx_casrn', con, if_exists='append', index=False, chunksize=65536, dtype=dtypes_cas)

In [13]:
# Check that the table contains expected data... 
cmd = text('select * from dtx_casrn limit 5;')
con.execute(cmd).fetchall()

[('26148-68-5', 'DTXSID7020001', 'A-alpha-C'),
 ('107-29-9', 'DTXSID2020004', 'Acetaldehyde oxime'),
 ('60-35-5', 'DTXSID7020005', 'Acetamide'),
 ('103-90-2', 'DTXSID2020006', 'Acetaminophen'),
 ('968-81-0', 'DTXSID7020007', 'Acetohexamide')]

In [14]:
nrows = con.execute(text('select count(*) from dtx_casrn;')).scalar()
print(nrows, 'DTXSID-CASRN mappings added to database')

719631 DTXSID-CASRN mappings added to database


### Load DTXSID:CID mappings

Each DTXSID is mapped onto one CID but non-uniquely (some share the same CID). Dropping SIDs entirely, to simplify the database.

In [15]:
dtx_cid_data = pd.read_table(DTX_CIDS, dtype=str)
dtx_cid_data.drop('SID', axis=1, inplace=True)
dtx_cid_data.drop_duplicates(inplace=True)  # Just to be safe

In [16]:
cid_cols = ['cid', 'dtxsid']
dtx_cid_data.columns = cid_cols
print(len(dtx_cid_data), 'DTXSID-CID mappings in data source')

735563 DTXSID-CID mappings in data source


In [17]:
# Filter mappings to include only DTXSIDs for which we already have a molecule.
dtx_cid_data = dtx_cid_data.loc[dtx_cid_data['dtxsid'].isin(ids['dtxsid'])]
print(len(dtx_cid_data), 'DTXSID-CID mappings available to add to database')

717020 DTXSID-CID mappings available to add to database


In [18]:
dtypes_cid = dict(zip(cid_cols, 2*[types.Text]))
dtx_cid_data.to_sql('dtx_cid', con, if_exists='append', index=False, chunksize=65536, dtype=dtypes_cid)

In [19]:
# Check that the table contains expected data... 
cmd = text('select * from dtx_cid limit 5;')
con.execute(cmd).fetchall()

[('25234023', 'DTXSID80873077'),
 ('139784', 'DTXSID90873068'),
 ('53316381', 'DTXSID40873033'),
 ('60196405', 'DTXSID80873032'),
 ('5282796', 'DTXSID60873030')]

In [20]:
nrows = con.execute(text('select count(*) from dtx_cid;')).scalar()
print(nrows, 'DTXSID-CID mappings added to database')

717020 DTXSID-CID mappings added to database


## Create view of all molecules and IDs

In [21]:
create_view = text(
    """
    CREATE MATERIALIZED VIEW compounds
    AS SELECT
        dsstox.dtxsid,
        dsstox.inchi,
        dsstox.inchikey,
        dsstox.molecule,
        dtx_cid.cid,
        dtx_casrn.casrn,
        dtx_casrn.name
    FROM dsstox
    LEFT OUTER JOIN dtx_cid ON dtx_cid.dtxsid = dsstox.dtxsid
    LEFT OUTER JOIN dtx_casrn ON dtx_casrn.dtxsid = dsstox.dtxsid;
    """
)
res = con.execute(create_view)
print(res.rowcount)

719631


In [22]:
con.execute(text('SELECT COUNT(*) FROM compounds where cid is null;')).scalar()

2611

In [23]:
con.execute(text('SELECT COUNT(*) FROM compounds where casrn is null;')).scalar()

0

In [24]:
# Check that the table contains expected data... 
cmd = text('select * from compounds limit 5;')
con.execute(cmd).fetchall()

[('DTXSID90873068', 'InChI=1S/C14H22O/c1-5-10(3)12-7-13(11(4)6-2)9-14(15)8-12/h7-11,15H,5-6H2,1-4H3', 'PFBHMJJMZWUEAI-UHFFFAOYSA-N', 'CCC(C)c1cc(O)cc(C(C)CC)c1', '139784', '14556-13-9', '3,5-Bis(1-methylpropyl)phenol'),
 ('DTXSID80873032', 'InChI=1S/C22H21Cl2F3N2O.ClH/c23-17-7-6-16(22(25,26)27)19(24)18(17)21(30)28-20(14-4-2-1-3-5-14)15-12-29-10-8-13(15)9-11-29;/h1-7,13,15,20H,8-12H2,(H,28,30);1H/t15-,20-;/m1./s1', 'QVUNQDRANLXLAJ-FOWJKZASSA-N', 'Cl.OC(=N[C@H](c1ccccc1)[C@@H]1CN2CCC1CC2)c1c(Cl)ccc(C(F)(F)F)c1Cl', '60196405', '1508278-73-6', 'N-[(S)-[(3S)-1-Azabicyclo[2.2.2]octan-3-yl](phenyl)methyl]-2,6-dichloro-3-(trifluoromethyl)benzamide-hydrogen chloride (1:1)'),
 ('DTXSID10872997', 'InChI=1S/C8H16/c1-3-5-7-8-6-4-2/h7-8H,3-6H2,1-2H3/b8-7+', 'IRUCBBFNLDIMIK-BQYQJAHWSA-N', 'CCC/C=C/CCC', '5357253', '14850-23-8', '(4E)-4-Octene'),
 ('DTXSID30872994', 'InChI=1S/C8H16/c1-3-5-7-8-6-4-2/h3,5H,4,6-8H2,1-2H3/b5-3+', 'ILPBINAXDRFYPL-HWKANZROSA-N', 'C/C=C/CCCCC', '5364448', '13389-42-9', '(2E)

## Create the index

Create an index on molecular structures using the GiST-powered RDKit extension. This is what enables substructure searching in SQL.

It takes a while...

In [25]:
cmd = text('CREATE INDEX molidx ON compounds USING gist(molecule);')
res = con.execute(cmd)